In [213]:
import pandas as pd
import numpy as np
import os
import sys
sys.path.append('../data/')

In [215]:
data_dirname = '../data/'
label_filename = 'labels.dat'
prefix = 'ex'

ex_dirnames = os.listdir(data_dirname)
ex_dirnames = filter(lambda x: x.startswith(prefix), ex_dirnames)
ex_dirnames = sorted(ex_dirnames)

csv_filenames = ['pos', 'neg']
skip_prefixes = ['Blank', 'blank', 'Flav', 'flav', 'Compound']



for k_ex in range(len(ex_dirnames)):
    current_dirname = ex_dirnames[k_ex]
    with open(data_dirname+current_dirname+'/'+label_filename, 'r') as f:
        labels = f.readlines()
        labels = list(map(lambda x: int(x[:-1]), labels))
    mz = []
    rt = []
    data = []
    samples = []
    for k_pol in range(len(csv_filenames)):
        current_filename = csv_filenames[k_pol]+'.csv'
        df = pd.read_csv(data_dirname+current_dirname+'/'+current_filename, skiprows=2)
        columns = df.columns.values
        columns = filter(lambda x: all([not x.startswith(y) for y in skip_prefixes]), columns)
        df = df[columns]
        mz.append(df.iloc[:, 0].values)
        rt.append(df.iloc[:, 1].values)
        data.append(df.values[:, 2:])
        samples.append(columns[2:])
    np.savez_compressed(
        data_dirname+current_dirname+'/'+current_dirname,
        mz=mz, rt=rt, samples=samples, data=data,
        csv_filenames=csv_filenames, labels=labels
    )   
    #del df

In [216]:
mza = 100.
mzb = 900.
mz_step = 1.

pos_ind_in = 0
neg_ind_in = (pos_ind_in+1) % 2
pos_ind_out = 1
neg_ind_out = (pos_ind_out+1) % 2


result = []
labels = []
for k_ex in range(len(ex_dirnames)):
    current_dirname = ex_dirnames[k_ex]
    df = np.load(data_dirname+current_dirname+'/'+current_dirname+'.npz')
    mz = df['mz']
    rt = df['rt']
    data = df['data']
    labels += df['labels'].tolist()
    posNpoints, posNsamples = data[pos_ind_in].shape
    negNpoints, negNsamples = data[neg_ind_in].shape
    for k_sample in range(posNsamples):
        local_data = np.zeros([int((mzb - mza)//mz_step), 2])
        for k_point in range(max(posNpoints, negNpoints)):
            if k_point < posNpoints:
                if (
                    (mza > mz[pos_ind_in][k_point]) or
                    (mzb <= mz[pos_ind_in][k_point]) or
                    (data[pos_ind_in][k_point, k_sample] == 0.)
                ):
                    continue
                    
                mzind = np.trunc((mz[pos_ind_in][k_point] - mza)/mz_step)
                mzind = int(mzind)
                local_data[mzind, pos_ind_out] = max(
                    local_data[mzind, pos_ind_out],
                    data[pos_ind_in][k_point, k_sample]
                )
            if k_point < negNpoints:
                if (
                    (mza > mz[neg_ind_in][k_point]) or
                    (mzb <= mz[neg_ind_in][k_point]) or
                    (data[neg_ind_in][k_point, k_sample] == 0.)
                ):
                    continue
                mzind = np.trunc((mz[neg_ind_in][k_point] - mza)/mz_step)
                mzind = int(mzind)
                local_data[mzind, neg_ind_out] = max(
                    local_data[mzind, neg_ind_out],
                    data[neg_ind_in][k_point, k_sample]
                )
        result.append(local_data)

result = np.array(result)

In [218]:
np.savez_compressed(data_dirname+'data.npz', data=result, labels=labels)

In [187]:
a, mean, std = mean_std(a, div = False)
ind = np.where(mean != 0)[1]

NameError: name 'mean_std' is not defined

In [219]:
dirnames = os.listdir(data_dirname)
dirnames = list(filter(lambda x: x.startswith('ex'), dirnames))
dirnames = sorted(dirnames)

uninteresting_column_prefixes = [
    'Flav_all', 'Blank', 'blank', 'Compound', 'Comp', 'm/z', 'Charge', 'Retention time (min)'
]

ex_dict = {
    'experiment': [],
    'label': [],
    'name_pos': [],
    'name_neg': []
}

labels_filename = 'labels.dat'


for dirname in dirnames:
    with open(data_dirname+dirname+'/'+labels_filename, 'r') as f:
        tmp_labels = f.readlines()
        tmp_labels = list(map(lambda x: x.replace('\n', ''), tmp_labels))
        tmp_labels = list(filter(lambda x: (len(x) > 0), tmp_labels))
        tmp_labels = list(map(int, tmp_labels))
        tmp_labels = list(filter(lambda x: x != -1, tmp_labels))
    ex_dict['label'] += tmp_labels
    ex_dict['experiment'] += [dirname]*len(tmp_labels)
    for pol in ['pos', 'neg']:
        df = pd.read_csv(data_dirname+dirname+'/'+pol+'.csv', skiprows=2)
        data_columns = [x for x in df.columns if not x.startswith(tuple(uninteresting_column_prefixes))]
        data_columns = list(map(lambda x: x.replace('#', '_'), data_columns))
        if dirname == 'ex1':
            data_columns = list(map(lambda x: 'pr'+x, data_columns))
        ex_dict['name_'+pol] += data_columns
exdf = pd.DataFrame(ex_dict)


In [220]:
def correctorCSV(x):
    result, tail = x.split('_Seg', 1)
    if 'ul_AutoMSMS' in result:
        result = 'pr'+result
    if 'G26' in result:
        result = result.replace('G26', 'G25')
    if '10rc_1k10_3' in result:
        result = result.replace('10rc', '10c')
    if ('c_1k10_' in result) and (result.endswith('r')):
        result = result[:-1]
    result += '_Seg' + tail
    return result

def corrector(x):
    result, tail = x.split('_Seg', 1)
    if '0_u' in result:
        result = result.replace('0_u', '0u')
    if '5_u' in result:
        result = result.replace('5_u', '5u')
    if (result == 'C52-1_13'):
        result = result[:-2] + '3'
    if (result == '15+_1k5_10ul_12'):
        result = result[:-2] + '2'
    if (result == '33_1k10_10ul_') or (result == '33h_1k10_10ul_'):
        result += '6'
    result += '_Seg' + tail
    return result

exdf['name_neg'] = exdf['name_neg'].apply(corrector)
exdf['name_pos'] = exdf['name_pos'].apply(corrector)

exdf['name_neg'] = exdf['name_neg'].apply(correctorCSV)
exdf['name_pos'] = exdf['name_pos'].apply(correctorCSV)
exdf.head()

,experiment,label,name_neg,name_pos
0,ex1,1,pr1_1k10_20ul_1_Seg1Ev2,pr1_1k10_20ul_1_Seg1Ev1
1,ex1,1,pr1_1k10_20ul_2_Seg1Ev2,pr1_1k10_20ul_2_Seg1Ev1
2,ex1,2,pr2_1k10_10ul_1_Seg1Ev2,pr2_1k10_10ul_1_Seg1Ev1
3,ex1,2,pr2_1k10_10ul_2_Seg1Ev2,pr2_1k10_10ul_2_Seg1Ev1
4,ex1,3,pr3_1k10_10ul_1_Seg1Ev2,pr3_1k10_10ul_1_Seg1Ev1


In [221]:
df = pd.read_csv(data_dirname+'s_plant_species_ident.txt', sep='\t')
df['Sample Name'] = df['Sample Name'].apply(corrector)
df.head()

,Source Name,Factor Value[year],Unit,Term Source REF,Term Accession Number,Characteristics[Organism],Term Source REF.1,Term Accession Number.1,Characteristics[Organism part],Term Source REF.2,Term Accession Number.2,Protocol REF,Sample Name,Factor Value[location],Term Source REF.3,Term Accession Number.3
0,no_treatment,2016,year,CCONT,http://purl.obolibrary.org/obo/UO_0000036,Melilotus officinalis,NCBITAXON,NaN,leaf,BTO,NaN,Sample collection,10-1_1_Seg1Ev1,Altai Krai,NaN,NaN
1,no_treatment,2016,year,CCONT,http://purl.obolibrary.org/obo/UO_0000036,Melilotus officinalis,NCBITAXON,NaN,leaf,BTO,NaN,Sample collection,10-1_1_Seg1Ev2,Altai Krai,NaN,NaN
2,no_treatment,2016,year,CCONT,http://purl.obolibrary.org/obo/UO_0000036,Melilotus officinalis,NCBITAXON,NaN,leaf,BTO,NaN,Sample collection,10-1_2_Seg1Ev1,Altai Krai,NaN,NaN
3,no_treatment,2016,year,CCONT,http://purl.obolibrary.org/obo/UO_0000036,Melilotus officinalis,NCBITAXON,NaN,leaf,BTO,NaN,Sample collection,10-1_2_Seg1Ev2,Altai Krai,NaN,NaN
4,no_treatment,2016,year,CCONT,http://purl.obolibrary.org/obo/UO_0000036,Melilotus officinalis,NCBITAXON,NaN,leaf,BTO,NaN,Sample collection,10-1_3_Seg1Ev1,Altai Krai,NaN,NaN


In [222]:
treatment_pos = list(
    map(
        lambda x: dict(name_pos=x, treatment=df[df['Sample Name'] == x]['Source Name'].values[0]),
        exdf.name_pos
    )
)

treatment_neg = list(
    map(
        lambda x: dict(name_neg=x, treatment=df[df['Sample Name'] == x]['Source Name'].values[0]),
        exdf.name_neg
    )
)

In [223]:
tmp_dfpos = pd.DataFrame(treatment_pos)
tmp_dfneg = pd.DataFrame(treatment_neg)

assert np.all(tmp_dfpos.treatment.values == tmp_dfneg.treatment.values)
assert np.all(tmp_dfpos.name_pos.values == exdf.name_pos.values)

exdf['treatment'] = tmp_dfpos.treatment.values
exdf.head()

,experiment,label,name_neg,name_pos,treatment
0,ex1,1,pr1_1k10_20ul_1_Seg1Ev2,pr1_1k10_20ul_1_Seg1Ev1,no_treatment
1,ex1,1,pr1_1k10_20ul_2_Seg1Ev2,pr1_1k10_20ul_2_Seg1Ev1,no_treatment
2,ex1,2,pr2_1k10_10ul_1_Seg1Ev2,pr2_1k10_10ul_1_Seg1Ev1,no_treatment
3,ex1,2,pr2_1k10_10ul_2_Seg1Ev2,pr2_1k10_10ul_2_Seg1Ev1,no_treatment
4,ex1,3,pr3_1k10_10ul_1_Seg1Ev2,pr3_1k10_10ul_1_Seg1Ev1,no_treatment


In [224]:
def corrector2(x):
    if x in [
        'pr1_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1',
        'pr16_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1',
        'pr18_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1',
        'pr19_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1',
        'pr35_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1',
        'pr20_1k10_20ul_AutoMSMS_Pos__1_Seg1Ev1'
    ]:
        return x.replace('_20ul_', '_10ul_')
    return x

exdf['name_pos'] = exdf['name_pos'].apply(corrector2)

In [225]:
def trimmer(x):
    result = x.split('_Seg')[0]
    if 'AutoMSMS' in x:
        result = x.split('_AutoMSMS')[0]
        result += '_AutoMSMS'
    if '0_u' in result:
        result = result.replace('0_u', '0u')
    if '5_u' in result:
        result = result.replace('5_u', '5u')
    if (result == 'C52-1_13'):
        result = result[:-2] + '3'
    if (result == '15+_1k5_10ul_12'):
        result = result[:-2] + '2'
    if (result == '33_1k10_10ul_') or (result == '33h_1k10_10ul_'):
        result += '6'
    return result

exdf['sample_pos'] = exdf['name_pos'].apply(trimmer)
exdf['trial_pos'] = exdf['sample_pos'].apply(lambda x: x[::-1].split('_', 1)[0][::-1])
exdf['sample_pos'] = exdf['sample_pos'].apply(lambda x: x[::-1].split('_', 1)[1][::-1])

exdf['sample_neg'] = exdf['name_neg'].apply(trimmer)
exdf['trial_neg'] = exdf['sample_neg'].apply(lambda x: x[::-1].split('_', 1)[0][::-1])
exdf['sample_neg'] = exdf['sample_neg'].apply(lambda x: x[::-1].split('_', 1)[1][::-1])

In [226]:
tmp = np.char.array(exdf['sample_neg'].values)
tmp = tmp + '_'
tmp = tmp + np.char.array(exdf['treatment'].values)
exdf['sample'] = tmp
print(len(np.unique(tmp)), len(exdf))
exdf.head()

#593 no_treatment

(187, 871)


,experiment,label,name_neg,name_pos,treatment,sample_pos,trial_pos,sample_neg,trial_neg,sample
0,ex1,1,pr1_1k10_20ul_1_Seg1Ev2,pr1_1k10_20ul_1_Seg1Ev1,no_treatment,pr1_1k10_20ul,1,pr1_1k10_20ul,1,pr1_1k10_20ul_no_treatment
1,ex1,1,pr1_1k10_20ul_2_Seg1Ev2,pr1_1k10_20ul_2_Seg1Ev1,no_treatment,pr1_1k10_20ul,2,pr1_1k10_20ul,2,pr1_1k10_20ul_no_treatment
2,ex1,2,pr2_1k10_10ul_1_Seg1Ev2,pr2_1k10_10ul_1_Seg1Ev1,no_treatment,pr2_1k10_10ul,1,pr2_1k10_10ul,1,pr2_1k10_10ul_no_treatment
3,ex1,2,pr2_1k10_10ul_2_Seg1Ev2,pr2_1k10_10ul_2_Seg1Ev1,no_treatment,pr2_1k10_10ul,2,pr2_1k10_10ul,2,pr2_1k10_10ul_no_treatment
4,ex1,3,pr3_1k10_10ul_1_Seg1Ev2,pr3_1k10_10ul_1_Seg1Ev1,no_treatment,pr3_1k10_10ul,1,pr3_1k10_10ul,1,pr3_1k10_10ul_no_treatment


In [227]:
labels_dict = {}
labels_unique = list(set(exdf.label.values))
exdf['label'] = exdf['label'].astype('i')
k = 0
for i in range(len(labels_unique)):
    label = labels_unique[i]
    tmp = exdf[exdf['label']==label]
    samples_unique = np.unique(tmp['sample'].values)
    cur_dict = {}
    for j in range(len(samples_unique)):
        ind = tmp[tmp['sample'] == samples_unique[j]].index.values
        cur_dict[samples_unique[j]] = ind
    labels_dict[label] = cur_dict


In [228]:
phys_labels = []
for i in range(len(labels_unique)):
    key = labels_unique[i]
    phys_labels += [key]*len(labels_dict[key])
print(len(phys_labels))

189


In [198]:
ind = []
for i in range(len(labels_unique)):
    ikey = labels_unique[i]
    jkeys = labels_dict[ikey].keys()
    for j in range(len(jkeys)):
        jkey = jkeys[j]
        ind += labels_dict[ikey][jkey].tolist()

In [229]:
uni, cnt = np.unique(phys_labels, return_counts=True)
np.mean(cnt), np.std(cnt, ddof=1)

(5.25, 0.7699721701835351)

In [230]:
data_filename = 'data.npz'

labels_init = np.load(data_dirname+data_filename)['labels']
labels_actual_unique = np.unique(labels_init)
def labelMapper(x):
    if x in labels_actual_unique:
        return x
    return -1

exdf['label_actual'] = exdf['label'].map(labelMapper)
labels_actual_dict = {}
for i in range(len(labels_actual_unique)):
    label = labels_actual_unique[i]
    tmp = exdf[exdf['label_actual']==label]
    samples_unique = np.unique(tmp['sample'].values)
    cur_dict = {}
    for j in range(len(samples_unique)):
        ind = tmp[tmp['sample'] == samples_unique[j]].index.values
        cur_dict[samples_unique[j]] = ind
    labels_actual_dict[label] = cur_dict


In [231]:
phys_labels_actual = []
sample_names = []
for i in range(len(labels_actual_unique)):
    key = labels_actual_unique[i]
    phys_labels_actual += [key]*len(labels_actual_dict[key])
    sample_names.append(labels_actual_dict[key].keys())
print(len(phys_labels_actual))

189


In [202]:
uni_actual, cnt_actual = np.unique(phys_labels_actual, return_counts=True)
np.mean(cnt_actual[1:]), np.std(cnt_actual[1:], ddof=1)

(5.257142857142857, 0.7800021547050611)

In [232]:
np.savez_compressed(
    data_dirname+'physical', phys_labels_actual=phys_labels_actual, labels_actual_dict=labels_actual_dict,
    sample_names=sample_names
)

In [203]:
snm = reduce(lambda x, y: x+y, sample_names)

In [233]:
from sklearn.model_selection import StratifiedShuffleSplit

n_splits = 100
random_state = 732
filename = 'physical_cv_indices_nc'
dirname = '../'

filename732 = 'phys_cv_indices_732'

y = phys_labels_actual
X = np.empty([len(y), 0])

splitter732 = StratifiedShuffleSplit(
    n_splits=n_splits,
    test_size=0.3,
    train_size=0.7,
    random_state=random_state
)

train_indices_732 = []
test_indices_732 = []

np.random.seed(random_state)
for train_index, test_index in splitter732.split(X, y):
    phys_train_index = []
    phys_test_index = []
    for i in range(max(len(train_index), len(test_index))):
        if i < len(train_index):
            train_ind = train_index[i]
            train_key = snm[train_ind]
            i_train = y[train_ind]
            phys_train_index += labels_actual_dict[i_train][train_key].tolist()
        if i < len(test_index):
            test_ind = test_index[i]
            test_key = snm[test_ind]
            i_test = y[test_ind]
            phys_test_index.append(
                np.random.choice(
                    labels_actual_dict[i_test][test_key]
                )
            )
    train_indices_732.append(phys_train_index)
    test_indices_732.append(phys_test_index)
    np.savez_compressed(
        data_dirname+filename732, n_splits=n_splits, random_state=random_state,
        train_indices=train_indices_732, test_indices=test_indices_732
    )

In [234]:
from sklearn.model_selection import StratifiedShuffleSplit

n_splits=100
random_state=235
filename='physical_cv_indices_nc'
dirname = '../'

filename73 = 'phys_cv_indices_73'

y = phys_labels_actual
X = np.empty([len(y), 0])



splitter73 = StratifiedShuffleSplit(
    n_splits=n_splits,
    test_size=0.3,
    train_size=0.7,
    random_state=random_state
)

train_indices_73 = []
test_indices_73 = []
np.random.seed(random_state)
for train_index, test_index in splitter73.split(X, y):
    phys_train_index = []
    phys_test_index = []
    for i in range(max(len(train_index), len(test_index))):
        if i < len(train_index):
            train_ind = train_index[i]
            train_key = snm[train_ind]
            i_train = y[train_ind]
            phys_train_index += labels_actual_dict[i_train][train_key].tolist()
        if i < len(test_index):
            test_ind = test_index[i]
            test_key = snm[test_ind]
            i_test = y[test_ind]
            phys_test_index.append(
                np.random.choice(
                    labels_actual_dict[i_test][test_key]
                )
            )
    train_indices_73.append(phys_train_index)
    test_indices_73.append(phys_test_index)
    np.savez_compressed(
        data_dirname+filename73, n_splits=n_splits, random_state=random_state,
        train_indices=train_indices_73, test_indices=test_indices_73
    )

array([array([[1.88098663e+04, 2.05901346e+04, 1.67983017e+04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.29878976e+04, 1.86094677e+04, 1.99356279e+04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.38628932e+04, 1.84165397e+04, 1.25672063e+04, ...,
        1.67415703e+06, 1.60054910e+06, 3.73496333e+04],
       ...,
       [0.00000000e+00, 2.86019156e+03, 6.44557159e+02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.88983822e+04, 1.93122989e+03, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.02155035e+03, 2.61638844e+03, 5.85470386e+03, ...,
        3.77380468e+03, 9.70493546e+03, 0.00000000e+00]]),
       array([[18507.06930542, 17680.01843262, 18617.47130122, ...,
            0.        ,     0.        , 71049.33077508],
       [ 3071.3618274 ,  1110.88223267,  5109.74963379, ...,
            0.        ,     0.        , 22906.65937441],
       [ 1725.12041164,     

In [182]:
_, c = np.unique(, return_counts=True)
np.mean(c)

5.25